# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [8]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [9]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [10]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-10-18&until-occurred-date=2021-10-25


In [11]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [12]:
pages = math.ceil(ed.events.getHits() / 1000)

17155 events found


In [13]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-18&until-occurred-date=2021-10-25
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=47bd5be6-ca90-456a-9107-2a7248c34fbf&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-18&until-occurred-date=2021-10-25
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=5d0f01df-5d15-45a3-a0b7-47a219e31e12&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-18&until-occurred-date=2021-10-25
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [14]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-10-18.csv
failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load preprint_tweets_2021-10-23.csv
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-11.csv
failed to load preprint_tweets_2021-10-04.csv
failed to load preprint_tweets_2021-10-06.csv
failed to load preprint_tweets_2021-10-07.csv
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [15]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [16]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264262...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264260...
REST API query complete  200
REST API query started for 10.1101/2021.09.13.21262182...
REST API query complete  200
REST API query started for 10.1101/2021.09.22.21263977...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.10.19.21265208...
REST API query complete  200
REST API query started for 10.1101/2021.02.16.21251535...
REST API query complete  200
REST API query started for 10.1101/2021.09.16.21263684...
REST API query complete  200
REST API query started for 10.1101/2021.09.30.462488...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.07.08

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.10.19.21265208,1705,medRxiv,Infectious Diseases (except HIV/AIDS),True,Airborne transmission of SARS-CoV-2 over dista...,"[{'name': 'Jennifer C Palmer'}, {'name': 'Daph...",<sec><title>Objective</title><p>To evaluate th...,2021-10-20
1,10.1101/2021.10.18.21264623,419,medRxiv,Infectious Diseases (except HIV/AIDS),True,SARS-CoV-2 variants associated with vaccine br...,"[{'name': 'Andrew D. Marques'}, {'name': 'Scot...",<p>The severe acute respiratory coronavirus-2 ...,2021-10-20
2,10.1101/2021.10.22.465012,96,bioRxiv,Neuroscience,False,Hippocampal spatio-temporal cognitive maps ada...,"[{'name': 'Mona M Garvert'}, {'name': 'Tankred...",<p>The brain forms cognitive maps of relationa...,2021-10-23
3,10.1101/2021.10.18.21265113,90,medRxiv,Infectious Diseases (except HIV/AIDS),True,Immune Responses in Fully Vaccinated Individua...,"[{'name': 'Ai-ris Y. Collier'}, {'name': 'Cath...",<sec><title>Background</title><p>A cluster of ...,2021-10-20
4,10.1101/2021.10.21.465374,41,bioRxiv,Neuroscience,False,Neural knowledge assembly in humans and deep n...,"[{'name': 'Stephanie M Nelli'}, {'name': 'Luka...",<p>Human understanding of the world can change...,2021-10-23
5,10.1101/2021.10.17.21265101,33,medRxiv,Epidemiology,True,Time-Varying Effectiveness of Three Covid-19 V...,"[{'name': 'Mónica M. Robles Fontán'}, {'name':...",<sec><title>Background</title><p>As of October...,2021-10-18
6,10.1101/2021.10.21.465323,26,bioRxiv,Genomics,False,Defining cellular complexity in human autosoma...,"[{'name': 'Yoshiharu Muto'}, {'name': 'Eryn E....",<p>Autosomal dominant polycystic kidney diseas...,2021-10-22
7,10.1101/2021.10.20.465093,26,bioRxiv,Genomics,False,Inherited MUTYH mutations cause elevated somat...,"[{'name': 'Philip S. Robinson'}, {'name': 'Lau...",<p>Cellular DNA damage caused by reactive oxyg...,2021-10-21
8,10.1101/2021.10.22.465484,25,bioRxiv,Evolutionary Biology,False,Genomic insights into variation in thermotoler...,"[{'name': 'Cheyenne Payne'}, {'name': 'Stephen...",<p>Understanding how organisms adapt to changi...,2021-10-24
9,10.1101/2021.10.20.465155,22,bioRxiv,Microbiology,False,Human milk oligosaccharides reduce murine grou...,"[{'name': 'Marlyd E Mejia'}, {'name': 'Samanth...",<p>Group B Streptococcus (GBS) colonizes the v...,2021-10-20


### Tweets of bioRxiv and medRxiv preprints

In [17]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

45 preprints (including 19 covering SARS-CoV-2, 27 from bioRxiv and 18 from medRxiv) published in the last 7 days before 2021-10-25 had been tweeted at least 3 times (maximum 1705).